<a href="https://colab.research.google.com/github/studyingstudent101/6031/blob/main/Data_603_Boyd_Hannah_Week15_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
#due last class 12/12
#might need multiple spark sessions?

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
#spark = SparkSession.builder.getOrCreate()

#spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "15g").getOrCreate()
#need memory for executor too? 4g?
spark = SparkSession.builder \
    .appName("RandomCrimeForest") \
    .config("spark.executor.memory", "15g") \
    .config("spark.driver.memory", "15g") \
    .getOrCreate()



from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler, OneHotEncoder, Imputer
#OneHotEncoderEstimator
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt


In [4]:
chicago_crime_file = r'/content/drive/MyDrive/603/Crimes_-_2001_to_Present_20231031.csv'
#might have to put booleans as strings. add _ for multiword names if necessary. below didn't work
#schema = "ID NUMBER, Case Number STRING, Date DATE, Block STRING, IUCR STRING, Primary Type STRING, Description STRING, Location Description STRING, Arrest BOOLEAN, Domestic BOOLEAN, Beat STRING, District STRING, Ward NUMBER, Community Area STRING, FBI Code STRING, X Coordinate NUMBER, Y Coordinate NUMBER, Year NUMBER, Updated On DATE, Latitude NUMBER, Longitude NUMBER, Location STRING"
#NUMBER isn't supported. need similar datatypes.
schema = "id INT, case_number STRING, date DATE, block STRING, iucr STRING, primary_type STRING, description STRING, location_description STRING, arrest BOOLEAN, domestic BOOLEAN, beat STRING, district STRING, ward INT, community_area STRING, fbi_code STRING, x_coordinate INT, y_coordinate INT, year INT, updated_on DATE, latitude DECIMAL, longitude DECIMAL, location STRING"

In [5]:
#loading after changing the NUMBER types to similar ones.
chicago_df = (spark
            .read
	        .format("csv")
	        .option("header", "true")
	        .schema(schema)
	        .load(chicago_crime_file))

In [6]:
#Got the setup
chicago_df.cache()

DataFrame[id: int, case_number: string, date: date, block: string, iucr: string, primary_type: string, description: string, location_description: string, arrest: boolean, domestic: boolean, beat: string, district: string, ward: int, community_area: string, fbi_code: string, x_coordinate: int, y_coordinate: int, year: int, updated_on: date, latitude: decimal(10,0), longitude: decimal(10,0), location: string]

In [7]:
chicago_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- case_number: string (nullable = true)
 |-- date: date (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: boolean (nullable = true)
 |-- domestic: boolean (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: integer (nullable = true)
 |-- community_area: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: integer (nullable = true)
 |-- y_coordinate: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- updated_on: date (nullable = true)
 |-- latitude: decimal(10,0) (nullable = true)
 |-- longitude: decimal(10,0) (nullable = true)
 |-- location: string (nullable = true)



In [8]:
#feature selection. iucr is what I'm trying to predict so leave that out
predictiveFeatures = ["block", "location_description", "beat", "district", "ward", "community_area", "location"]
featureHold = chicago_df.select(predictiveFeatures)

In [9]:
#Maybe dropping the columns is what caused issues originally? Focus on columns I'm using, see if I can remove rest later
other_df = chicago_df.na.drop(subset = predictiveFeatures)

In [10]:
#got 'string indices must be integers' error in next cell. will this fix it?
futureFeature = []
#turn the features I'm using into their own list
for att in predictiveFeatures:
    futureFeature.append({'predictive attributes': att})


In [11]:
pd.DataFrame(futureFeature)

,predictive attributes
0,block
1,location_description
2,beat
3,district
4,ward
5,community_area
6,location


In [12]:
#can I index numerical cols too through string indexer?
#do i need to get through each distinct value in each column? would make sense for conversion
#index the features
for att in futureFeature:
    indexer = StringIndexer(inputCol=att['predictive attributes'], outputCol='%s_indexed' % att['predictive attributes'])
    string_model = indexer.fit(other_df)
    other_df = string_model.transform(other_df)

In [13]:
#target variable. puts it back with the other attributes
label_indexer = StringIndexer(inputCol='primary_type', outputCol='primary_type_indexed')
label_fit = label_indexer.fit(other_df)
other_df = label_fit.transform(other_df)

In [14]:
#make the indexed predictive features a column again for future inputs. putting for in front isn't working. can do it after like past attempts at making this run?
index_all = ['%s_indexed' % full['predictive attributes'] for full in futureFeature]

In [15]:
vectors_assemble = VectorAssembler(inputCols = index_all, outputCol = 'features')
other_vector = vectors_assemble.transform(other_df)

In [16]:
trainingSet, testingSet = other_vector.randomSplit([0.8, 0.2], seed = 1234)

In [17]:
#Logistic Regression
#works at this point
from pyspark.ml.classification import LogisticRegression

crime_regression = LogisticRegression(labelCol='primary_type_indexed', featuresCol='features', maxIter=10, family='multinomial')

In [18]:
crime_regression

LogisticRegression_6acc53cceed1

In [19]:
#sometimes has an error when fitting. looks like a connection issue.
crime_fitted = crime_regression.fit(trainingSet)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: ignored

In [ ]:
#Works, looks like it was just weird runtime errors before
crime_fitted

In [ ]:
#evaluating the attempt
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [ ]:
pred = crime_fitted.transform(testingSet)

eval = MulticlassClassificationEvaluator(labelCol = "primary_type_indexed", predictionCol = "prediction", metricName= "accuracy")
accuracy = eval.evaluate(pred)
#can keep previous parameters
precision = eval.evaluate(pred, {eval.metricName: "weightedPrecision"})
recall = eval.evaluate(pred, {eval.metricName: "weightedRecall"})
f1score = eval.evaluate(pred, {eval.metricName: "f1"})

print("Logistic Accuracy:", accuracy)
print("Logistic Precision:", precision)
print("Logistic Recall:", recall)
print("Logistic F1 Score:", f1score)


In [ ]:
#haven't tested this one yet
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="primary_type_indexed", featuresCol="features")

rfFit = rf.fit(trainingSet)

predTwo = rfFit.transform(testingSet)

In [ ]:
evalTwo = MulticlassClassificationEvaluator(labelCol = "primary_type_indexed", predictionCol = "prediction", metricName= "accuracy")
accuracyTwo = evalTwo.evaluate(predTwo)
precisionTwo = evalTwo.evaluate(pred, {evalTwo.metricName: "weightedPrecision"})
recallTwo = evalTwo.evaluate(pred, {evalTwo.metricName: "weightedRecall"})
f1scoreTwo = evalTwo.evaluate(pred, {evalTwo.metricName: "f1"})

print("Random Accuracy:", accuracyTwo)
print("Random Precision:", precisionTwo)
print("Random Recall:", recallTwo)
print("Random F1 Score:", f1scoreTwo)



In [ ]:
matrix_df = chicago_df.groupBy("arrest").pivot("domestic").agg(
    sum(when(chicago_df["domestic"] == True, 1).otherwise(0)).alias("Domestic"),
    sum(when(chicago_df["domestic"] == False, 1).otherwise(0)).alias("Non-Domestic")).orderBy("arrest")

matrix_df.show()

In [ ]:
#Pie Chart of Crime Type
pieDF = chicago_df.groupBy("iucr").count().orderBy(desc("count")).limit(10)

type_data = pieDF.collect()
types = [row['iucr'] for row in type_data]
counts = [row['count'] for row in type_data]

plt.figure(figsize=(8, 6))
plt.pie(counts, labels=types, autopct='%1.1f%%')
plt.title('Top 10 Crimes')
plt.axis('equal')
plt.show()

In [ ]:
#Bar Chart of Locations
loc_counts = chicago_df.groupBy('location_description').count().orderBy(desc('count')).limit(10)


location_count =loc_counts.collect()
ltypes = [row['location_description'] for row in location_count]
lcounts = [row['count'] for row in location_count]

plt.figure(figsize=(10, 6))
plt.bar(ltypes, lcounts, color='skyblue')
plt.xlabel('location_description')
plt.ylabel('count')
plt.title('Top 10 Crime Locations')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
year_totals = chicago_df.groupBy('year').count()

year_collection = year_totals.collect()

# Separate years and counts for plotting
years = [row['year'] for row in year_collection]
counts = [row['count'] for row in year_collection]

plt.figure(figsize=(10, 6))
plt.bar(years, counts, color='blue')
plt.xlabel('Years')
plt.ylabel('Count')
plt.title('Crime Years')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
spark.stop()